Manuall install `structlog`

`pip install structlog`

In [1]:
# %load_ext autoreload
# %autoreload 2
import os

os.environ["BASETEN_WEBHOOK_SECRET"] = "<your_webhook_secret>"
os.environ["BASETEN_API_KEY"] = "<your_baseten_api_key>"
os.environ["BASETEN_GPU"] = "A10G"

In [2]:
# from flow_judge.models.adapters.baseten.deploy import ensure_model_deployment

# ensure_model_deployment()

In [3]:
# Read the sample data
import json
from flow_judge import EvalInput

with open("sample_data/csr_assistant.json", "r") as f:
    data = json.load(f)

# Create a list of inputs and outputs
inputs_batch = [
    [
        {"query": sample["query"]},
        {"context": sample["context"]},
    ]
    for sample in data
]
outputs_batch = [{"response": sample["response"]} for sample in data]

# Create a list of EvalInput
eval_inputs_batch = [EvalInput(inputs=inputs, output=output) for inputs, output in zip(inputs_batch, outputs_batch)]

WARNING 10-18 21:39:55 _custom_ops.py:18] Failed to import from vllm._C with ModuleNotFoundError("No module named 'vllm._C'")
INFO 10-18 21:39:55 importing.py:10] Triton not installed; certain GPU-related functions will not be available.


In [4]:
from flow_judge.metrics import RESPONSE_FAITHFULNESS_5POINT
from flow_judge.utils.prompt_formatter import format_user_prompt, format_vars, format_rubric

metric = RESPONSE_FAITHFULNESS_5POINT

def format_prompt(eval_input: EvalInput) -> str:
    """Format the prompt for a single evaluation input."""
    prompt_variables = {
        "INPUTS": format_vars(eval_input.inputs),
        "OUTPUT": format_vars([eval_input.output]),
        "EVALUATION_CRITERIA": metric.criteria,
        "RUBRIC": format_rubric(metric.rubric),
    }
    return format_user_prompt(prompt_variables)

prompts = [format_prompt(eval_input) for eval_input in eval_inputs_batch]
prompts

['# GOAL\nYour job is to evaluate a task carried out by an AI system powered by a large language model.\n\nYou will be provided with the inputs and output of the task, as well as the evaluation criteria and scoring rubric. Your task is to evaluate the output of the AI system based on the evaluation criteria and scoring rubric provided.\n\n# INPUT\nBelow are the inputs required for performing the task:\n<inputs>\n<query>\nPlease read the technical issue that the user is facing and help me create a detailed solution based on the context provided.\n</query>\n<context>\n# Customer Issue: I\'m having trouble pushing large files to my Git repository. It says \'File size exceeds GitHub\'s file size limit of 100 MB\'.\n\n# Context: Git Large File Storage (LFS) replaces large files such as audio samples, videos, datasets, and graphics with text pointers inside Git, while storing the file contents on a remote server like GitHub.com or GitHub Enterprise.\n\nTo use Git LFS, you need to download an

In [5]:
from flow_judge.models.adapters.baseten.adapter import AsyncBasetenAPIAdapter

adapter = AsyncBasetenAPIAdapter(
    model_id="<model_id>",
    webhook_proxy_url="https://proxy.flowrite.com"
)

await adapter._async_fetch_batched_response(prompts)
# await adapter._async_fetch_response(prompts[0])


2024-10-18 21:39:58 [info     ] ACTIVE                        
2024-10-18 21:39:58 [info     ] HERE                          
2024-10-18 21:40:00 [debug    ] Batch 0: [{'index': 1, 'prompt': '# GOAL\nYour job is to evaluate a task carried out by an AI system powered by a large language model.\n\nYou will be provided with the inputs and output of the task, as well as the evaluation criteria and scoring rubric. Your task is to evaluate the output of the AI system based on the evaluation criteria and scoring rubric provided.\n\n# INPUT\nBelow are the inputs required for performing the task:\n<inputs>\n<query>\nPlease read the technical issue that the user is facing and help me create a detailed solution based on the context provided.\n</query>\n<context>\n# Customer Issue: I\'m having trouble pushing large files to my Git repository. It says \'File size exceeds GitHub\'s file size limit of 100 MB\'.\n\n# Context: Git Large File Storage (LFS) replaces large files such as audio samples, vid

INFO:flow_judge.models.adapters.baseten.validation:Baseten signature is valid!
INFO:flow_judge.models.adapters.baseten.validation:Baseten signature is valid!
INFO:flow_judge.models.adapters.baseten.validation:Baseten signature is valid!
INFO:flow_judge.models.adapters.baseten.validation:Baseten signature is valid!
INFO:flow_judge.models.adapters.baseten.validation:Baseten signature is valid!
INFO:flow_judge.models.adapters.baseten.validation:Baseten signature is valid!


BatchResult(successful_outputs=[{'id': '356439656307a30d5c62ef11445df762', 'index': 1, 'prompt': '# GOAL\nYour job is to evaluate a task carried out by an AI system powered by a large language model.\n\nYou will be provided with the inputs and output of the task, as well as the evaluation criteria and scoring rubric. Your task is to evaluate the output of the AI system based on the evaluation criteria and scoring rubric provided.\n\n# INPUT\nBelow are the inputs required for performing the task:\n<inputs>\n<query>\nPlease read the technical issue that the user is facing and help me create a detailed solution based on the context provided.\n</query>\n<context>\n# Customer Issue: I\'m having trouble pushing large files to my Git repository. It says \'File size exceeds GitHub\'s file size limit of 100 MB\'.\n\n# Context: Git Large File Storage (LFS) replaces large files such as audio samples, videos, datasets, and graphics with text pointers inside Git, while storing the file contents on 